In [43]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import feather
import scipy as sp 
from time import time
from sklearn.manifold import spectral_embedding
from sklearn.cluster import spectral_clustering
from sklearn.cluster.spectral import SpectralClustering
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import *
import pickle
from sklearn.utils.graph import graph_laplacian
from sklearn.utils.arpack import eigsh
from sklearn.manifold.spectral_embedding_ import _set_diag
from scipy.linalg import eigvals, eigvalsh
import seaborn
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.grid_search import GridSearchCV
from sklearn import tree
from sklearn.feature_selection import SelectKBest
from sklearn.cross_validation import train_test_split



%matplotlib inline

In [41]:
def test_clf(grid_search, features, labels, parameters, iterations=100):
    from sklearn.metrics import classification_report
    precision, recall = [], []
    for iteration in range(iterations):
        features_train, features_test, labels_train, labels_test = train_test_split(features, labels, random_state=iteration)
        grid_search.fit(features_train, labels_train)
        predictions = grid_search.predict(features_test)
    print classification_report(labels_test, predictions)
    best_params = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print '%s=%r, ' % (param_name, best_params[param_name])

In [23]:
label = pd.read_csv('../data/recomm_spectral_clustering_discretize.label.csv', index_col='biz_id')
precom = pd.read_csv('../data/tfidf_all_biz_1percom.csv', index_col='biz_id')

In [26]:
for biz in label.index:
    precom.loc[biz, 'label' ] = label.label.loc[biz]

In [39]:
precom_na = precom.dropna()
features = precom_na.drop('label', axis=1).values
labels = precom_na.label.values

In [46]:
clf = tree.DecisionTreeClassifier()
t0= time()
parameters = {'criterion': ['gini', 'entropy'],
               'min_samples_split': [2, 10, 20],
               'max_depth': [None, 2, 5, 10],
               'min_samples_leaf': [1, 5, 10],
               'max_leaf_nodes': [None, 5, 10, 20]}
grid_search = GridSearchCV(clf, parameters)
for i in [2,3,4,5]:
    k_best = SelectKBest(k=i)
    k_features = k_best.fit_transform(features, labels)
    print "K best features with k = {0}".format(i)
    test_clf(grid_search, k_features, labels, parameters)

print '\nDecision Tree  (Best k features):'

print '\nDecision Tree total time:{0}s'.format(round(time()-t0, 3))

K best features with k = 2


KeyboardInterrupt: 

In [48]:
from sklearn.ensemble import AdaBoostClassifier
from time import time
clf = AdaBoostClassifier()
t0= time()
parameters = {'n_estimators': [10, 20, 40],
               'algorithm': ['SAMME', 'SAMME.R'],
               'learning_rate': [.5, 1, 1.5]}
grid_search = GridSearchCV(clf, parameters)
for i in [2,3,4,5]:
    k_best = SelectKBest(k=i)
    k_features = k_best.fit_transform(features, labels)
    print "K best features with k = {0}".format(i)
    test_clf(grid_search, k_features, labels, parameters)

print '\nAdaBoost (Best k features):'

print '\nAdaBoost total time:{0}s'.format(round(time()-t0, 3))

K best features with k = 2


/home/sam/.virtualenvs/ist558Project/local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

        0.0       0.85      1.00      0.92      6750
        1.0       0.00      0.00      0.00       264
        2.0       0.29      0.03      0.05       356
        3.0       0.00      0.00      0.00       314
        4.0       0.50      0.01      0.02       291

avg / total       0.75      0.85      0.78      7975

algorithm='SAMME.R', 
learning_rate=1, 
n_estimators=20, 
K best features with k = 3
             precision    recall  f1-score   support

        0.0       0.85      1.00      0.92      6750
        1.0       0.00      0.00      0.00       264
        2.0       0.33      0.00      0.01       356
        3.0       0.00      0.00      0.00       314
        4.0       0.00      0.00      0.00       291

avg / total       0.73      0.85      0.78      7975

algorithm='SAMME.R', 
learning_rate=0.5, 
n_estimators=20, 
K best features with k = 4
             precision    recall  f1-score   support

        0.0       0.85    

In [50]:
k_features.shape

(31897, 5)